In [86]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as pofs
import xlsxwriter
import warnings

warnings.filterwarnings('ignore')

In [3]:
stocks = pd.read_csv('./sp_500_stocks.csv')
token = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [13]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={token}'
data = requests.get(api_url).json()

In [39]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

final_df = pd.DataFrame(columns=my_columns)

In [40]:
for symbol_string in symbol_strings: 
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            final_df = final_df.append(pd.Series([symbol, 
                                       data[symbol]['price'], 
                                       data[symbol]['stats']['year1ChangePercent'], 
                                       'N/A'], index=my_columns), ignore_index=True)
        except:
            pass

In [43]:
final_df.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df = final_df.reset_index(drop=True)

In [140]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('Not a Number')
        portfolio_size = input('Enter the size of your portfolio: ')

In [141]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [142]:
position_size = float(portfolio_size) / len(final_df.index)
for i in range(len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price'])

In [143]:
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,82.75,2.342979,241
1,OXY,71.01,1.102589,281
2,MCK,364.05,0.835536,54
3,DVN,72.62,0.824194,275
4,HRB,42.3,0.772316,472
5,MRO,29.22,0.722383,684
6,XOM,108.08,0.703143,185
7,CF,104.02,0.700886,192
8,MPC,112.2,0.685,178
9,VRTX,301.25,0.681449,66


In [144]:
hqm_columns = [
    'Ticker', 
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
]
hqm_df = pd.DataFrame(columns=hqm_columns)

In [145]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={token}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        try:
            hqm_df = hqm_df.append(pd.Series([
                symbol, 
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
            ], index=hqm_columns),
            ignore_index=True)
        except:
            pass

In [146]:
hqm_df.dropna(inplace=True)

In [147]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

In [148]:
for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row, percentile_col] = pofs(hqm_df[change_col], hqm_df.loc[row, change_col])/100

In [149]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,133.85,N/A,-0.145112,0.49497,0.07353,0.901408,0.155206,0.887324,0.021253,0.794769
1,AAL,14.43,N/A,-0.311099,0.195171,-0.26284,0.189135,-0.060355,0.372233,-0.003795,0.633803
2,AAP,178.99,N/A,-0.19163,0.410463,-0.192957,0.334004,-0.04976,0.394366,0.048467,0.903421
3,AAPL,149.45,N/A,-0.013834,0.702213,-0.129125,0.531187,-0.021627,0.468813,-0.048227,0.392354
4,ABBV,145.87,N/A,0.413439,0.943662,-0.075093,0.655936,-0.024216,0.458753,0.014426,0.7666
...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.52,N/A,-0.10467,0.559356,-0.086631,0.633803,-0.056914,0.382294,-0.037716,0.454728
497,ZBH,109.03,N/A,-0.252897,0.293763,-0.129878,0.529175,0.057049,0.71831,-0.030735,0.496982
498,ZBRA,270.08,N/A,-0.507977,0.040241,-0.339664,0.080483,-0.108402,0.203219,-0.08676,0.227364
499,ZION,50.82,N/A,-0.14782,0.484909,-0.176119,0.394366,0.029761,0.625755,-0.10337,0.160966


In [150]:
from statistics import mean

for row in hqm_df.index:
    mom_percentiles = []
    for time_period in time_periods:
        mom_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(mom_percentiles)

In [151]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,133.85,N/A,-0.145112,0.49497,0.07353,0.901408,0.155206,0.887324,0.021253,0.794769,0.769618
1,AAL,14.43,N/A,-0.311099,0.195171,-0.26284,0.189135,-0.060355,0.372233,-0.003795,0.633803,0.347586
2,AAP,178.99,N/A,-0.19163,0.410463,-0.192957,0.334004,-0.04976,0.394366,0.048467,0.903421,0.510563
3,AAPL,149.45,N/A,-0.013834,0.702213,-0.129125,0.531187,-0.021627,0.468813,-0.048227,0.392354,0.523642
4,ABBV,145.87,N/A,0.413439,0.943662,-0.075093,0.655936,-0.024216,0.458753,0.014426,0.7666,0.706237
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,114.52,N/A,-0.10467,0.559356,-0.086631,0.633803,-0.056914,0.382294,-0.037716,0.454728,0.507545
497,ZBH,109.03,N/A,-0.252897,0.293763,-0.129878,0.529175,0.057049,0.71831,-0.030735,0.496982,0.509557
498,ZBRA,270.08,N/A,-0.507977,0.040241,-0.339664,0.080483,-0.108402,0.203219,-0.08676,0.227364,0.137827
499,ZION,50.82,N/A,-0.14782,0.484909,-0.176119,0.394366,0.029761,0.625755,-0.10337,0.160966,0.416499


In [152]:
hqm_df.sort_values(by='HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True, inplace=True)

In [153]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MPC,114.25,N/A,0.698731,0.987928,0.223851,0.985915,0.286261,0.977867,0.132185,0.987928,0.984909
1,UNM,43.1,N/A,0.635386,0.977867,0.339593,0.995976,0.356881,0.98994,0.078988,0.961771,0.981388
2,LB,81.69,N/A,2.343236,1.0,0.839977,1.0,0.224795,0.947686,0.080201,0.967807,0.978873
3,COP,122.59,N/A,0.690958,0.985915,0.176123,0.977867,0.41713,0.995976,0.058508,0.923541,0.970825
4,COG,22.45,N/A,0.350433,0.937626,0.183511,0.979879,0.257516,0.961771,0.259466,0.997988,0.969316
5,CAH,72.3,N/A,0.543627,0.967807,0.146554,0.969819,0.30535,0.985915,0.066984,0.937626,0.965292
6,XOM,107.11,N/A,0.688905,0.983903,0.164767,0.975855,0.188867,0.927565,0.082094,0.969819,0.964286
7,LW,87.65,N/A,0.567883,0.971831,0.289873,0.991952,0.176514,0.909457,0.119429,0.981891,0.963783
8,DVN,72.55,N/A,0.836691,0.995976,0.122909,0.955734,0.289653,0.979879,0.045965,0.897384,0.957243
9,PSX,101.01,N/A,0.248534,0.901408,0.127445,0.957746,0.180124,0.915493,0.199779,0.993964,0.942153


In [154]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [155]:
position_size = float(portfolio_size) / len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i, 'Price'])

In [156]:
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MPC,114.25,175,0.698731,0.987928,0.223851,0.985915,0.286261,0.977867,0.132185,0.987928,0.984909
1,UNM,43.1,464,0.635386,0.977867,0.339593,0.995976,0.356881,0.98994,0.078988,0.961771,0.981388
2,LB,81.69,244,2.343236,1.0,0.839977,1.0,0.224795,0.947686,0.080201,0.967807,0.978873
3,COP,122.59,163,0.690958,0.985915,0.176123,0.977867,0.41713,0.995976,0.058508,0.923541,0.970825
4,COG,22.45,890,0.350433,0.937626,0.183511,0.979879,0.257516,0.961771,0.259466,0.997988,0.969316
5,CAH,72.3,276,0.543627,0.967807,0.146554,0.969819,0.30535,0.985915,0.066984,0.937626,0.965292
6,XOM,107.11,186,0.688905,0.983903,0.164767,0.975855,0.188867,0.927565,0.082094,0.969819,0.964286
7,LW,87.65,228,0.567883,0.971831,0.289873,0.991952,0.176514,0.909457,0.119429,0.981891,0.963783
8,DVN,72.55,275,0.836691,0.995976,0.122909,0.955734,0.289653,0.979879,0.045965,0.897384,0.957243
9,PSX,101.01,198,0.248534,0.901408,0.127445,0.957746,0.180124,0.915493,0.199779,0.993964,0.942153


In [157]:
writer = pd.ExcelWriter('momentnum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, 'Momentnum Strategy', index = False)

In [158]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)
percentile_format = writer.book.add_format(
    {
        'num_format': '0.00%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [159]:
column_formats = {
    'A': ['Ticker', string_format], 
    'B': ['Price', dollar_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One-Year Price Return', percentile_format],
    'E': ['One-Year Return Percentile', percentile_format],
    'F': ['Six-Month Price Return', percentile_format],
    'G': ['Six-Month Return Percentile', percentile_format],
    'H': ['Three-Month Price Return', percentile_format],
    'I': ['Three-Month Return Percentile', percentile_format],
    'J': ['One-Month Price Return', percentile_format],
    'K': ['One-Month Return Percentile', percentile_format],
    'L': ['HQM Score', percentile_format],
}

for col in column_formats.keys():
    writer.sheets['Momentnum Strategy'].set_column(f'{col}:{col}', 25, column_formats[col][1])
    writer.sheets['Momentnum Strategy'].write(f'{col}1', column_formats[col][0], column_formats[col][1])
writer.save()

In [139]:
writer.close()